In [1]:
import nltk
import numpy as np
import pandas as pd
from pandas import DataFrame
from keras.models import Model, Sequential, load_model
from keras.preprocessing.sequence import TimeseriesGenerator, pad_sequences
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.core import Activation, Dropout, Dense, Flatten
from keras.layers import TimeDistributed, Bidirectional, InputLayer, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import TensorBoard
from keras.metrics import categorical_accuracy
from IPython.display import clear_output
from more_itertools import flatten, intersperse
import random

import os
import urllib.request

Using TensorFlow backend.
/Users/toniga/miniconda3/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [35]:
%load_ext autoreload
%autoreload 2
from batcher import batch_from_generator
from train_data import load_conll2003, create_conll_encoded_shifted_generator
from mappings import get_all_mappings
from corpus import corpus_training_data_generator, create_all_corpus_train_pipeline, pad, encode_each_sentence
from model import create_model, compile_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
DROPOUT = 0.2
TIME_SLICE_SIZE = 128
BATCH_SIZE = 512
SAMPLING_RATE = 1
PADDING = 0
UNKNOWN = 1
NUM_OF_UNITS = 250
WORDS_PER_BATCH = 100
EPOCHS=1
MODEL_SAVE_PATH = 'tc_model.h5'
LSTM_MODEL_SAVE_PATH = 'lstm_model.h5'

In [4]:
mapping, reverse_mapping, lower_mapping, lower_reverse_mapping = get_all_mappings()

In [5]:
# def create_model(shape, classes):
#     # First layer inputs must be 3D
#     # with shape (samples, timesteps, features)
#     model = Sequential()
#     model.add(InputLayer(input_shape=shape))
#     model.add(Bidirectional(LSTM(NUM_OF_UNITS, return_sequences=True, dropout=DROPOUT, recurrent_dropout=DROPOUT)))
#     model.add(TimeDistributed(Dense(1000)))
#     model.add(TimeDistributed(Dense(classes)))
#     model.add(Activation('softmax'))
#     return model

In [6]:
model = create_model(len(mapping), NUM_OF_UNITS, DROPOUT)

In [7]:
model = compile_model(model)

In [8]:
if os.path.isfile(MODEL_SAVE_PATH): 
    model.load_weights(MODEL_SAVE_PATH)

In [9]:
tensor_board = TensorBoard(batch_size=BATCH_SIZE, write_graph=False, )

In [10]:
X, Y = create_all_corpus_train_pipeline(TIME_SLICE_SIZE)

In [11]:

#g = corpus_training_data_generator('gutenberg',TIME_SLICE_SIZE, BATCH_SIZE, shift=False)
model.fit(X, Y,  verbose=2, validation_split=0.2, callbacks=[tensor_board], batch_size=200, epochs=1)

Train on 37666 samples, validate on 9417 samples
Epoch 1/1
 - 369s - loss: 0.3138 - acc: 0.9469 - val_loss: 0.1712 - val_acc: 0.9568


In [55]:
test_sentence = pad(["I want to go on holiday to France".lower()], TIME_SLICE_SIZE)
test_sentence = encode_each_sentence(test_sentence, mapping)

mapped_sentence = np.asarray(test_sentence)
predicted_result = model.predict_classes(mapped_sentence)[0]
predicted_result = list(zip(pad(["I want to go on holiday to France".lower()], TIME_SLICE_SIZE)[0], predicted_result.tolist()))

def true_case(letter, label):
    if letter == 0:
        return ''
    if(label == 2):
        return letter.lower()
    if (label == 3):
        return letter.upper()
    if (label == 0):
        return ''
    return letter

predicted_result = [true_case(letter, label) for letter, label in predicted_result]
''.join(predicted_result) 



'i want to go on holiday to france'

In [13]:
model.save("big_lstm.h5")